In [ ]:
import numpy as np
import pandas as pd
from glob import glob
from netCDF4 import Dataset
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from skimage.graph import route_through_array

In [ ]:
# Load Sea/Land fixed data to create a mask
nc_f = 'data/sftlf_ARC-22_CCCma-CanESM2_rcp45_r1i1p1_CCCma-CanRCM4_r2_fx.nc'
nc = Dataset(nc_f, 'r')
#nc.variables.keys()
sftlf = nc['sftlf'][0, :, :]

In [ ]:
# get historical transit years
hist_transits = pd.read_html('http://www.nauticapedia.ca/Articles/NWP_Fulltransits.php')[0]
# skip where transit was only by an icebreaker, stress easier travels
no_break = hist_transits[~hist_transits['Vessel Type'].isna()]
no_break = no_break[~no_break['Vessel Type'].str.contains('break')]
no_break_transit_years = no_break.Year.str.split().str.get(0)
no_break_transit_years = no_break_transit_years[no_break_transit_years.str.isalnum()].astype(np.int32).unique()

In [ ]:
# files downloaded from http://climate-modelling.canada.ca/climatemodeldata/canrcm/CanRCM4/ARC-22_CCCma-CanESM2_historical/day/atmos/sic/index.shtml
#hist_files = glob('data/historic/day/1/*.nc')
hist_files = glob('data/historic/day/*.nc')

In [ ]:
%%time

# process years

weights = []
indices_len = []
times = []

for hist_file in hist_files:
    
    nc = Dataset(hist_file, 'r')
    print(hist_file, nc['time'].shape)
    times.extend(list(nc['time'][:]))
    
    for i in range(nc.variables['time'].shape[0]):
        sic = nc['sic'][i, :, :]
        sic[np.where(sftlf!=0)] = -np.inf
        indices, weight = route_through_array(sic, [0,116], [265,231], geometric=True, fully_connected=True)
        weights.append(weight)
        indices_len.append(len(indices))

In [ ]:
df = pd.DataFrame([weights, indices_len, times]).T
df.columns = ['weights', 'num_index', 'time']
df['date'] = pd.to_datetime(df.time, unit='D', origin='1949-12-01')
df['year'] = df['date'].dt.year
df['dayofyear'] = df.date.dt.dayofyear
df['no_break_transit'] = df.year.isin(no_break_transit_years)

In [ ]:
df.sort_values('date', inplace=True)

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
ax.scatter(df[df.no_break_transit==False].date, df[df.no_break_transit==False].weights, label='Year no transit')
ax.scatter(df[df.no_break_transit==True].date, df[df.no_break_transit==True].weights, label='Year transit')
ax.set_ylim(0.0001, 50000)
ax.set_yscale('log')
ax.set_ylabel('MCP weight', labelpad=10, fontsize=20)
ax.set_xlabel('Days', labelpad=10, fontsize=20)
plt.legend()
plt.savefig('plots/transit_years.png', dpi=72, pad_inches=0, bbox_inches="tight")
plt.show()

In [ ]:
from scipy.stats import ks_2samp, ttest_ind

In [ ]:
%%time

test_res = []
for x in np.arange(0, 60, 0.5):
    res = []
    for k,g in df.groupby('year'):
        res.append([k, (g.weights<x).sum() / len(g), g.no_break_transit.iloc[0]])
    res = pd.DataFrame(res, columns=['year', 'days_per_year_under', 'no_break_transit'])
    t = res.loc[res.no_break_transit==True, 'days_per_year_under']
    f = res.loc[res.no_break_transit==False, 'days_per_year_under']
    tt_stat, tt_pval = ttest_ind(t, f)
    ks_stat, ks_pval = ks_2samp(t,f)
    test_res.append([x, tt_stat, tt_pval, ks_stat, ks_pval])
test_res = pd.DataFrame(test_res, columns=['thresh', 'tt_stat', 'tt_pval', 'ks_stat', 'ks_pval'])

In [ ]:
%%time

day_res = []
for x in np.arange(0, 60, 0.5):
    res = []
    for k,g in df.groupby('year'):
        res.append([k, (g.weights<x).sum(), g.no_break_transit.iloc[0]])
    res = pd.DataFrame(res, columns=['year', 'days_under', 'no_break_transit'])
    t = res.loc[res.no_break_transit==True, 'days_under']
    f = res.loc[res.no_break_transit==False, 'days_under']
    day_res.append([x, np.median(t), np.median(f)])
day_res = np.array(day_res)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(test_res.thresh, test_res['tt_pval'], label='t-test')
ax.plot(test_res.thresh, test_res['ks_pval'], label='ks-test')
ax.set_ylabel('P value', labelpad=10, fontsize=20)
ax.set_xlabel('Threshold', labelpad=10, fontsize=20)
ax.set_yscale('log')
plt.legend()
plt.savefig('plots/threshold_test.png', dpi=72, pad_inches=0, bbox_inches="tight")
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(8,6))
ax.plot(day_res[:, 0], day_res[:,1], label='transit')
ax.plot(day_res[:, 0], day_res[:,2], label='non-transit')
ax.set_ylabel('Median Days Under Threshold', labelpad=10, fontsize=20)
ax.set_xlabel('Threshold', labelpad=10, fontsize=20)
plt.legend(loc="lower right")
plt.savefig('plots/threshold_days.png', dpi=72, pad_inches=0, bbox_inches="tight")
plt.show()